In [2]:
import pandas as pd
from feast import FeatureStore, FileSource, ValueType, Entity, Feature, FeatureView

# Cargar los datos en un DataFrame de Pandas
users_df = pd.read_csv("users.csv")

# Crear una fuente de datos en Feast que apunte al archivo CSV
users_source = FileSource(
    path="path/to/users.yaml",
    event_timestamp_column="event_timestamp",
    created_timestamp_column="created_timestamp"
)

# Crear una entidad para los usuarios
user_entity = Entity(name="user", value_type=ValueType.INT64, description="Un usuario en el sistema")

# Crear una característica para la edad de los usuarios
age_feature = Feature(name="age", dtype=ValueType.INT32)

# Crear una vista de características para los usuarios
users_view = FeatureView(
    name="users",
    entities=["user"],
    ttl="7d",
    features=[age_feature],
    online=True,
    input=user_source
)

# Cargar los datos en el Feature Store
store = FeatureStore(repo_path="my_project")
store.update_feature_view(users_view, users_df)


ModuleNotFoundError: No module named 'feast'